In [1]:
import torch
import glove
import run
import pickle
from tools import Dictionary

wandb: Currently logged in as: ayushi15. Use `wandb login --relogin` to force relogin


In [64]:
EMBEDDING_SIZE = 128
CONTEXT_SIZE = 3
NUM_EPOCH = 100
BATHC_SIZE = 512
PATH = "../../checkpoints/vanilla"

In [65]:
# !mv /Users/v/Downloads/glove.pt model/

In [66]:
with open("../../data/dict_test_vanilla.pkl", mode='rb') as fp:
    dictionary = pickle.load(fp)
    
VS = dictionary.vocab_size

In [67]:
# doc

In [68]:
model = glove.GloVeModel(EMBEDDING_SIZE, CONTEXT_SIZE, VS)
model.load_state_dict(torch.load(PATH, map_location=torch.device('cpu')))
model
# model = torch.load(PATH, map_location=torch.device('cpu'))

GloVeModel(
  (_focal_embeddings): Embedding(21237, 128)
  (_context_embeddings): Embedding(21237, 128)
  (_focal_biases): Embedding(21237, 1)
  (_context_biases): Embedding(21237, 1)
)

In [69]:
print(model._focal_embeddings.weight.sum())
print(model._context_embeddings.weight.sum())

tensor(1916.3533, dtype=torch.float64, grad_fn=<SumBackward0>)
tensor(-3045.9561, dtype=torch.float64, grad_fn=<SumBackward0>)


In [70]:
emb = model._focal_embeddings.weight + model._context_embeddings.weight
# emb = model['_focal_embeddings.weight'] + model['_context_embeddings.weight']

emb = [e.detach().numpy() for e in emb]
print(emb[0].sum())
print(emb[1].sum())
print(emb[2].sum())

8.574258117652906
2.8309328196182237
-1.249055585696773


In [71]:
print(dictionary.word2idx['hogwarts'])
print(dictionary.word2idx['dumbledore'])
print(dictionary.word2idx['voldemort'])

2164
719
836


In [72]:
GloVe = {}
for w in dictionary.word2idx:
    GloVe[w] = emb[dictionary.word2idx[w]]
    
    
print(GloVe['hogwarts'].sum())
print(GloVe['dumbledore'].sum())
print(GloVe['voldemort'].sum())

1.6786581138087002
2.103210732167823
-0.5495063313950732


In [73]:
import numpy as np

def cosine(a, b):
    return a.dot(b)/(np.linalg.norm(a)*np.linalg.norm(b))

In [82]:
cosine(GloVe['harry'], GloVe['dursley'])

-0.02034403650795824

In [37]:
cosine(GloVe['hogwarts'], GloVe['harry'])

-0.08967246662227145

In [ ]:
# with open("model/glove.pkl", "wb") as handle:
#     pickle.dump(GloVe, handle)

# Analogies

In [38]:
from utils import find_analogy_glove

In [85]:
embeddings = GloVe
word_a = "harry"
word_b = "potter"
word_c = "luna"

# Trevor toad Fluffy dog

#    Lovegood


In [86]:
%%time
find_analogy_glove(word_a, word_b, word_c, embeddings, 20)

CPU times: user 182 ms, sys: 18.6 ms, total: 200 ms
Wall time: 193 ms


'hall'